<a href="https://colab.research.google.com/github/mvdheram/AHCVS_Notes/blob/main/Color_cubes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Color cubes

Paper : https://homes.cs.washington.edu/~bradc/cv/pubs/conferences/gi96.pdf

**What ?**
  * **Approximation algorithm** 
    * *It is a hierarchical method that **accelerates the rendering process** without greatly sacrificing image quality.* [https://homes.cs.washington.edu/~bradc/cv/pubs/conferences/gi96.pdf]

**Why ?**

* **Handle scenes with partially transparant group of objects occluding each other** 
  * Level of details and occlusion culling approaches does not work
  * E.g. :
    * **Objects arranged in straight line with partial occulusion**
* How to render when
  * **Number (objects) > Number (trianlges)**  ??
* **Accelerate rendering of complex scenes partially transparant group of objects.**

   

**How ?**

  * Group large number of similar objects hierarchically as cluster
  * Replace the cluster with substitute/replacement - color cubes
    * Render the approximate color and opacity (color cube) of object rather than triangles if projected size on image plane is small to reduce the rendering cost. 
    ---  
  * Details :
      * **Pre-procesing**
        * Uses : Octree data structure
          * Subdivide the scene using octree 
          * Each node in the hierarchy/octree represents a region of the scene, or cell.   
          * Associated with each octree cell is an
  approximation using a color-cube — 
            * Color-cube : A cube with a color and opacity associated with each of its six faces 
      * **Rendering**  
        * This approximation can be rendered in less time than the geometry within the cell, and
is used in place of the geometry when the projected size
of the cell on the image plane is sufficiently small. [https://homes.cs.washington.edu/~bradc/cv/pubs/conferences/gi96.pdf

    ---



## Color cubes approach 



**Approximation approach** 
1. Group similar groups of objects into cluster 
  * Clustering can be done hierarchically (clustering group of similar cluster)
2. Replace the cluster 
  * Choices
    * Replace entire group by images(texture)
      * Replace the object with texture 
        * *Texture : Texture mapping is the process of taking a 2D image and mapping onto a polygon in the scene. This texture acts like a painting, adding 2D detail to the 2D polygon.* 
    * Compute new polygon meshes (approximated polygon)
      * *Polygon meshes : Collection of lines, vetices, faces that define an object.*  

**Color cubes approach :**

* **Pre-processing** 
  1. Construct 3D octree subdivision of input scene.
    * All triangles stored in leaves
    * Overlapping children (cutting boundary of octree cells)
      * Store several times in all overlapping children (can be handled by GPUs)
  2. Store color cube (same size as the octree cell) in each octree cell (inner nodes). Each side of color cube has a color and opacity value. 
    * **Color-cube**
      * Replace the cluster of objects of an area by approximated colored cube 
      * Each side of color cube consists of 
        * Color 
        * Opacity (transparency value)
          * 1.0 opaque
          * 0.0 transparant 
    * **Why color cube?**
      * The color and transparency approximate an orthogonal view of the triangles stored in the cell.
      * **Computed in pre-processing**
* **Rendering** 
  3. Traverse the octree to render the scene 
  ```
    Render(octree cell)
      
      if (no octree cell overlap frustum)
        * return
      
      if projected size of octree cell <= threshold 
        * Render color cube of the octree cell
      
      else if octree cell is leaf 
        * Render the trianlges of the cell 
          * projected size can be > threshold, but render triangles as they are leaf nodes not inner nodes 
      
      else (projected size > threshold)
        for all children of the octree cell(back to front)
        * Render(child) - Recurse 
  ```

---

* Threshold typically set to no longer than one pixel 
  * Why ?
    * Color cube approximation only spans a pixel
    * Larger values of
threshold can also be used to render lower-quality images in less
time.
  * Larger values of threshold means not going deep into tree, so no need to render triangles

* Projected size of octree cell depends on the view point
  * The **projected size (leaf nodes of octree nearer to view point frustum == inner nodes of octree farther from view point frustum)**.
  * Hence, color cubes are rendered for leaf nodes nearer to frustum and color cubes of inner nodes rendered for inner nodes farther from frustum
    * Rendering color cubes saves the rendering time as child nodes are not visited for color cube rendered octree cell.

---

**Pre-processing phase**

* Construction of 
  * Octree and distribution of trianlges
    * Start with root node containing entire trianlges of scene.
    * Recursively subdivide the root scene and distrubute triangles into children node.
      * Stopping condition for subdivision 
        * Determine experimentally with the number of triangle to be stored in leaves as estimate.  
  * Color cubes (children -> parents)
    1. Leaves (initial)
      * **Why ?**
        * Color cubes of inner nodes are based on leave nodes
      * **How ?**
        * For 6 sides of octree cell 
          * Compute color and opacity per side of octree cell
            * Render the image (Rasterization) of the different views/sides of octree cell  by orthographic projection.
            * Compute 
                * Color value  : Compute average of the rastered pixel of the image
                * Opcity value : percent of pixe of the face
            * Save the alpha (opacity) value along with color value
            ---
            * Orthographic projection :
              * Projection of 3D object to 2D screen
              * Parallel projection based on view point 
                * Project the object based on view point (6 view points front,rear, left.. for cube) onto projection plane 
            * Rasterization : 
              * Render object by placing the objects onto pixel grid and color the pixel the object covers by testing the distance/depth value (z-value) of object and pixel.
                
                Algorithm :
                * if (depth(object) < depth(pixel) )
                  * Color(pixel with object)
                * else
                  * Discard object  
            ---
  
    2. Internal nodes 
      * Calculate color-cube based on color cubes of children
      * **How ?**
        * For all six sides of octree cell : 
          * **Calculate the pairwise weighted average (based on opacity values alpha of children color cube) of color values of the 8 children to get color cube of parent node.**
          ---
          1. Piece-wise average of color value taking into account transparancy value alpha (8 child color cubes) -> 4 color cubes
          2. Average of color values (4 color cube) -> single color cube (parent)
          ---
            * **How?**
              * Considering the front side, calculate piece-wise average of front and back color cube values. 
              * Results in color cube having 4 children after piece wise average of 8 children
              * Average the 4 color values of color cube to produce single color for each side.  




## Runtime analysis 

* **Rendering algorithm has run time of O(log(n))**
Two assumption 

1. Depth of octree cell d = O(log(n))
2. The maximum number of geometric primitives per leaf cell is constant.

**Proof**
* Divide the frustum into 
  * Close area (>threshold): Triangles are rendered 
    * Assumption that there are constant number of cells O(1)
  * Wide area (<threshold):  Color cubes are rendered 
    * Sub frustum corresponding to levels of octree (i)
      * Sub frustum is divided into 
        * d, d-1, d-2 ..
        * d - lowest depth of the octree ( render color cubes of leave nodes)
        * As depth increases, color cubes of inner nodes are rendered
    * As we can render only a constant number of color cubes based on threshold value 
    * With assumption that depth of octree cell is O(log(n))
    * **The rendering time is O(log(n))**
  * Frustum tests:
    * The number of cells to be tested to cull cells which are not in the frustum
    * Takes O(log(n))
  * **Overall O(log(n))**

           

